In [ ]:
import boto3
from boto3.dynamodb.conditions import Key
import json
from decimal import Decimal
from mingus.core import scales, notes, chords

class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return float(obj)
        return super(DecimalEncoder, self).default(obj)

flat_to_sharp_mapping = {
    'C': 'C', 'Db': 'C#', 'D': 'D', 'Eb': 'D#', 'E': 'E', 'F': 'F',
    'Gb': 'F#', 'G': 'G', 'Ab': 'G#', 'A': 'A', 'Bb': 'A#', 'B': 'B',
    'Fb': 'E', 'Cb': 'B'
}

def get_roman_numeral(scale, chord_root, quality):
    roman_numerals = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']
    position = scale.index(chord_root)
    roman_numeral = roman_numerals[position]
    return roman_numeral if quality == 'MAJOR' else roman_numeral.lower()

def update_chord_info(chord_info, roman_numeral):
    updated_info = chord_info.copy()
    updated_info['Nashville Number'] = roman_numeral
    return updated_info

def create_scale(root, scale_type):
    if scale_type == 'major':
        return scales.Major(root)
    elif scale_type == 'natural_minor':
        return scales.NaturalMinor(root)
    elif scale_type == 'dorian':
        return scales.Dorian(root)
    elif scale_type == 'harmonic_minor':
        return scales.HarmonicMinor(root)
    elif scale_type == 'melodic_minor':
        return scales.MelodicMinor(root)
    elif scale_type == 'mixolydian':
        return scales.Mixolydian(root)        
    else:
        raise ValueError(f"Unsupported scale type: {scale_type}")

def create_scale_with_sharps(root, scale_type):
    scale_obj = create_scale(root, scale_type)
    scale_notes = scale_obj.ascending()
    
    sharp_pitches = []
    for note in scale_notes:
        if 'b' in note:
            sharp_equivalent = flat_to_sharp_mapping.get(note, note)
            sharp_pitches.append(sharp_equivalent)
        else:
            sharp_pitches.append(note)
    
    return sharp_pitches

sharp_to_flat_mapping = {v: k for k, v in flat_to_sharp_mapping.items() if k != v}

dynamodb = boto3.resource('dynamodb')
table_name = 'ChordLib'
table = dynamodb.Table(table_name)

def query_chord(root):
    sharp_root = flat_to_sharp_mapping.get(root, root)
    response = table.query(
        KeyConditionExpression=Key('Root').eq(sharp_root)
    )
    items = response['Items']
    
    print(f"Query result for {sharp_root}: {items}")
    return items


def get_scale_data(key, scale_type='major'):
    scale = create_scale_with_sharps(key, scale_type)

    result = {
        "key": key,
        "scale_type": scale_type,
        "scale": scale,
        "chords": []
    }

    print(f"Scale: {scale}")

    for chord in scale:
        chord_data = query_chord(chord)
        print(f"Chord data for {chord}: {chord_data}")
        
        if chord_data:
            updated_variations = []
            for variation in chord_data:
                quality = variation['Quality']
                roman_numeral = get_roman_numeral(scale, chord, quality)
                updated_chord_info = update_chord_info(variation['ChordInfo'], roman_numeral)
                variation['ChordInfo'] = updated_chord_info
                updated_variations.append(variation)
            
            result["chords"].append({
                "root": sharp_to_flat_mapping.get(chord, chord),
                "variations": updated_variations
            })
        else:
            result["chords"].append({
                "root": sharp_to_flat_mapping.get(chord, chord),
                "variations": []
            })

    print(f"Final result: {result}")
    return result

    print(f"Scale: {scale}")

    for chord in scale:
        chord_data = query_chord(chord, fetch_all, max_variations)
        print(f"Chord data for {chord}: {chord_data}")
        if chord_data:
            result["chords"].append({
                "root": sharp_to_flat_mapping.get(chord, chord),
                "variations": chord_data
            })
        else:
            result["chords"].append({
                "root": sharp_to_flat_mapping.get(chord, chord),
                "variations": []
            })

    print(f"Final result: {result}")
    return result

def lambda_handler(event, context):
    try:
        params = event.get('queryStringParameters', {})
        key = params.get('key', 'C')
        scale_type = params.get('scale_type', 'major')

        result = get_scale_data(key, scale_type)

        return {
            'statusCode': 200,
            'body': json.dumps(result, cls=DecimalEncoder),
            'headers': {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            }
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)}),
            'headers': {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            }
        }